<h1> DISCLAIMER</H1>

This code was published with the purpose of training my programming skills on python and solving an old and recurring request from some users of the site about exporting their bookshelves.

This code is not intended to make a deep scraping on the site, but only allow the user collect their books from the shelf for themselves.

<h1>About Skoob (via Wikipedia):</h1>

Skoob is a collaborative social network for Brazilian readers, launched in January 2009 by developer Lindenberg Moreira.Without advertising, the site became a meeting point for readers and writers who exchange tips about books and organize meetings in bookstores. The network allows interaction with other social networks, such as Facebook and Twitter, as well as popular e-commerce stores in Brazil, such as: Americanas.com, Saraiva, and Submarino.

<h1>About Selenium</h1>

I'm using Selenium for scraping this site. For more information for to how install and how to use it, please see:
https://www.selenium.dev/ or https://www.geeksforgeeks.org/browser-automation-using-selenium/



In [1]:
# importing packages
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup as bs
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
from pandas import DataFrame as df
start_time = time()

from warnings import warn

url="https://www.skoob.com.br/login/"
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)



In [9]:
#login
usr= input('usuário')
psw= input('senha')

#localizando as áreas de login:
username = driver.find_element_by_id("UsuarioEmail")
password = driver.find_element_by_id("UsuarioSenha")

username.send_keys(usr)
password.send_keys(psw)
sleep(2)

driver.find_element_by_xpath('//*[@id="login-box-col02"]/form/div/div/input').click()

sleep(5)
print("Login Ok")


When you enter in your page after login, the  address it's like https://www.skoob.com.br/usuario/ID-FIRSTNAME

And your bookshelf is in: https://www.skoob.com.br/estante/livros/todos/ID

So, we need some regular expression to collect the ID-Number from initial page.



In [10]:

id_number= re.findall('[0-9]+',driver.current_url)
id_number=str(id_number[0])

# url refresh 
url= "https://www.skoob.com.br/estante/livros/todos/" + id_number
driver.get(url)
driver.refresh
sleep(3)

#we need use the cascate layout to scraping the books, so
driver.find_element_by_xpath('/html/body/div/div[2]/div[2]/div/div/div[2]/div/div[1]/a[2]').click()

print("Sucssefull screen ajustment ")

In [61]:
#I need to calculate the numbers of pages because just clicking on next button results on a infinite loop.
qty_books=driver.find_element_by_xpath('//*[@id="corpo"]/div/div[4]/div[2]/div[1]/div[1]/div/span[1]').text
qty_books= re.findall("[0-9]+", qty_books)
qty_pages=round(int(qty_books[0])/36)+1
qty_pages

#I'm sure that there are a easiest way, but this solve my problem for while


WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.88)


In [55]:
#list of books
l_books=[]

#counter
qty=1


while qty<=qde_pages:
    books=driver.find_elements_by_class_name('clivro')
    for b in books:
        d=b.find_element_by_class_name('livro-conteudo')
        title= d.find_element_by_class_name('ng-binding').text
        author_editora=d.find_element_by_tag_name('p').text
        #this approach returns one string with two informations, so we go use str.split() funtion to solve. We'll use the new line character (\n) as delimiter
        author_publisher=  author_publisher.split('\n')
        #agora posso separar
        author=author_publisher[0]
        publisher= author_publisher[1]
        b_details={'title':title,'author': author,'publisher': publisher}          
        l_books.append(b_details)
    qty+=1
    # next page
    driver.find_element_by_xpath('//*[@id="corpo"]/div/div[4]/div[2]/div[1]/div[2]/ul/li[8]/a').click()
    sleep(3)


len(l_books)

806

In [60]:
#saving the file
dict = l_books
df_books = pd.DataFrame(dict, index=None)  

arcv="my_books.csv"
     
df_books.to_csv(arcv)

len(df_books)


806